## Libraries

In [32]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error
from pathlib import Path

In [33]:
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [34]:
from xgboost import XGBRegressor

## Load data

In [35]:
boston = load_boston()

# Initializing the dataframe
data = pd.DataFrame(boston.data)

#Adding the feature names to the dataframe
data.columns = boston.feature_names
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [36]:
#Adding target variable to dataframe
data['PRICE'] = boston.target 

In [37]:
# Spliting target variable and independent variables
X = data.drop(['PRICE'], axis = 1)
y = data['PRICE']

In [38]:
# Matrix with feature transformation
# X = pd.read_csv("newBoston.csv")

In [39]:
# Splitting to training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 4)

In [40]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Optimization function

In [41]:
def opt_bas(n_estimators, max_depth, eta):
    
    # parameters
    hyperparameters = {
        "n_estimators" : int(n_estimators),
        "max_depth" : int(max_depth),
        "eta" : eta,
        "objective": "reg:squarederror",
        "random_state": 1,
    }
    
    # fit and predict
    model = XGBRegressor(**hyperparameters).fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    # get score
    metric = mean_absolute_percentage_error(y_test, y_pred)*100
    
    return -metric

## Run optimization

In [42]:
class newJSONLogger(JSONLogger):

      def __init__(self, path):
            self._path=None
            super(JSONLogger, self).__init__()
            self._path = path if path[-5:] == ".json" else path + ".json"

In [47]:
# Bounded region of parameter space
pbounds = {'n_estimators': (40, 80), 'max_depth': (1, 7), 'eta': (0.05, 0.5)}

# Bayes optimizer instantiation
optimizer = BayesianOptimization(f=opt_bas, 
                                 pbounds=pbounds, 
                                 random_state=2, verbose=2, 
                                )

# keep data
log_path = Path().resolve() / "Logs" / "Boston_mape_leo.json"
logger = newJSONLogger(path = str(log_path))
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [48]:
optimizer.maximize(init_points=8, n_iter=400)